In [29]:
!pip install numpy
!pip install pandas
!pip install scikit-network

You should consider upgrading via the '/home/danial/PycharmProjects/MIR-HWs3-5/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/danial/PycharmProjects/MIR-HWs3-5/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 8.5 MB 5.3 MB/s eta 0:00:01
You should consider upgrading via the '/home/danial/PycharmProjects/MIR-HWs3-5/venv/bin/python -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np

# Dataset:
https://www.kaggle.com/datasets/hwassner/TwitterFriends?resource=download

In [85]:
import ast

df = pd.read_csv("./data/data.csv", sep=',(?=\S)', engine="python")
print("df read done!")
df['tags'] = df['tags'].apply(ast.literal_eval)
df['friends'] = df['friends'].apply(lambda x: list(map(lambda x: int(x.strip('", ')), x.strip("[] ").split())))
df['id'] = df['id'].apply(lambda x: int(x.strip('", ')))
df['followersCountRank'] = df['followersCount'].rank(ascending=False)

df read done!


In [71]:
df.columns, df.shape, df.dtypes

(Index(['id', 'screenName', 'tags', 'avatar', 'followersCount', 'friendsCount',
        'lang', 'lastSeen', 'tweetId', 'friends', 'followersCountRank'],
       dtype='object'),
 (40000, 11),
 id                      int64
 screenName             object
 tags                   object
 avatar                 object
 followersCount          int64
 friendsCount            int64
 lang                   object
 lastSeen                int64
 tweetId                object
 friends                object
 followersCountRank    float64
 dtype: object)

In [15]:
graph_edges = []
def add_edges(row):
    for friend in row['friends']:
        graph_edges.append((row['id'], friend))

df.apply(lambda row: add_edges(row), axis=1)
graph_edges[:10]

[(1969527638, 1969574754),
 (1969527638, 1969295556),
 (1969527638, 1969284056),
 (1969527638, 1969612214),
 (1969527638, 1970067476),
 (1969527638, 1969797386),
 (1969527638, 1969430539),
 (1969527638, 1969840064),
 (1969527638, 1969698176),
 (1969527638, 1970005154)]

In [16]:
from sknetwork.data import from_edge_list

graph = from_edge_list(graph_edges)

In [17]:
from sknetwork.ranking import PageRank
pagerank = PageRank()
scores = pagerank.fit_transform(graph.adjacency)

In [89]:
df_scores = pd.concat([df.set_index('id'), pd.DataFrame({"id": graph.names, "score": scores}).set_index('id')], join="inner", axis=1)
df_scores.sort_values(by=["score"], ascending=False).head(30)[['screenName', 'tags', 'followersCount', 
                                                               'friendsCount', 'followersCountRank', 'score']]

,screenName,tags,followersCount,friendsCount,followersCountRank,score
id,,,,,,
1906208438,"""nourraanahmed""",[#nationaldogday],100419,104158,164.0,0.000399
9721292,"""abc15""",[#nationaldogday],246053,65677,65.0,0.000366
604723016,"""AngelaxHoran""",[#nationaldogday],30711,20933,432.0,0.000366
4369196414,"""JillVasbinder22""",[#nationaldogday],93087,34056,176.0,0.000353
80145924,"""AlyssaDotCom""",[#nationaldogday],88530,87381,185.0,0.000352
582683927,"""Kobes_Lakers""",[#narcos],53152,42480,291.0,0.000345
718799630,"""FactsInYourFace""",[#nationaldogday],133369,87434,133.0,0.000344
177378345,"""kentonjduty""",[#nationaldogday],415344,23292,37.0,0.000338
15304100,"""10TV""",[#nationaldogday],217873,25184,80.0,0.000338


In [91]:
from sknetwork.ranking import Katz
katz = Katz()
scores = katz.fit_transform(graph.adjacency)
df_scores = pd.concat([df.set_index('id'), pd.DataFrame({"id": graph.names, "score": scores}).set_index('id')], join="inner", axis=1)
df_scores.sort_values(by=["score"], ascending=False).head(30)[['screenName', 'tags', 'followersCount', 
                                                               'friendsCount', 'followersCountRank', 'score']]

,screenName,tags,followersCount,friendsCount,followersCountRank,score
id,,,,,,
16190898,"""RyanSeacrest""",[#nationaldogday],14765554,645,1.0,8.921936e+10
23497233,"""Sia""",[#nationaldogday],2544982,253,9.0,5.369246e+10
2856237754,"""itsdanielbtw""",[#respecttylerjoseph],13743,14919,911.0,4.664591e+10
58309829,"""NickelodeonTV""",[#nationaldogday],3981436,2457,3.0,4.516935e+10
384047557,"""colliscool""",[#respecttylerjoseph],134730,757,132.0,4.458856e+10
54387096,"""producerdeb""",[cape canaveral],8765,9352,1339.5,4.140997e+10
2720648282,"""davistri3""",[#respecttylerjoseph],1076,3273,8582.0,4.086500e+10
26642006,"""Alyssa_Milano""",[#nationaldogday],2887813,1537,6.0,4.073731e+10
2208137546,"""_nicole27""",[#nationaldogday],565,5001,16053.5,3.827654e+10
